In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
subject = 'Acre - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [7]:
data = pd.read_csv('2003_01_model_input_AC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)', 'Acre - Produção de Cimento (t)'], axis=1)
data

,Ano,Acre - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - Consumo de Cimento (t)
0,2003,0.691320,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,21.091289,...,22.694889,17.485235,79.741292,1.381276,8.722906,7.285707e+06,413409.255032,9.318417,6.587220e+06,43.603000
1,2004,0.694888,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,21.518782,...,23.048861,17.975141,79.766708,1.449721,8.664365,7.395794e+06,415792.317375,9.337292,6.624626e+06,58.142000
2,2005,0.698098,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,22.206186,...,23.627338,18.815686,79.914689,1.554670,8.632024,7.499439e+06,415180.374027,9.359959,6.658363e+06,80.914000
3,2006,0.700957,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,23.194739,...,24.284937,19.786299,80.112364,1.645156,8.552716,7.593871e+06,418006.330823,9.385224,6.685538e+06,114.415000
4,2007,0.703473,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,24.525685,...,24.876274,20.666407,80.286864,1.670211,8.542429,7.683282e+06,420042.950889,9.415550,6.712707e+06,121.872000
5,2008,0.705654,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,26.240265,...,25.255963,21.235435,80.365321,1.578870,8.514659,7.755114e+06,422290.202070,9.424555,6.724366e+06,119.145000
6,2009,0.707507,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,27.667957,...,25.376339,21.219990,79.673793,1.436413,8.542386,7.805166e+06,424173.603186,9.409042,6.722105e+06,121.146000
7,2010,0.709040,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,30.587558,...,25.288855,17.738219,70.049719,1.246421,8.514295,7.991788e+06,416829.675918,9.394384,6.762279e+06,174.705000
8,2011,0.710262,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,34.156125,...,25.242893,16.188303,66.211701,1.234468,8.500790,8.164220e+06,410997.875281,9.386906,6.795421e+06,178.322000
9,2012,0.711004,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,31.678793,...,25.454002,17.919229,70.364429,1.472205,10.334544,8.325627e+06,407346.950074,9.383420,6.827686e+06,173.494000


In [8]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Acre - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
0,0.691320,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,21.091289,982.380945,1230.453116,22.694889,17.485235,79.741292,1.381276,8.722906,7.285707e+06,413409.255032,9.318417,6.587220e+06
1,0.694888,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,21.518782,982.725729,1242.845968,23.048861,17.975141,79.766708,1.449721,8.664365,7.395794e+06,415792.317375,9.337292,6.624626e+06
2,0.698098,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,22.206186,983.285492,1262.499386,23.627338,18.815686,79.914689,1.554670,8.632024,7.499439e+06,415180.374027,9.359959,6.658363e+06
3,0.700957,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,23.194739,983.923087,1282.761558,24.284937,19.786299,80.112364,1.645156,8.552716,7.593871e+06,418006.330823,9.385224,6.685538e+06
4,0.703473,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,24.525685,984.501367,1296.980528,24.876274,20.666407,80.286864,1.670211,8.542429,7.683282e+06,420042.950889,9.415550,6.712707e+06
5,0.705654,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,26.240265,984.883184,1298.504336,25.255963,21.235435,80.365321,1.578870,8.514659,7.755114e+06,422290.202070,9.424555,6.724366e+06
6,0.707507,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,27.667957,984.971935,1299.760655,25.376339,21.219990,79.673793,1.436413,8.542386,7.805166e+06,424173.603186,9.409042,6.722105e+06
7,0.709040,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,30.587558,985.037587,1305.345781,25.288855,17.738219,70.049719,1.246421,8.514295,7.991788e+06,416829.675918,9.394384,6.762279e+06
8,0.710262,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,34.156125,985.282207,1317.710216,25.242893,16.188303,66.211701,1.234468,8.500790,8.164220e+06,410997.875281,9.386906,6.795421e+06
9,0.711004,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,31.678793,985.498854,1297.603435,25.454002,17.919229,70.364429,1.472205,10.334544,8.325627e+06,407346.950074,9.383420,6.827686e+06


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      58.142000
1      80.914000
2     114.415000
3     121.872000
4     119.145000
5     121.146000
6     174.705000
7     178.322000
8     173.494000
9     193.672000
10    171.709286
11    151.486629
12    134.223086
13    150.491000
14    148.696000
15    153.806000
16    178.033000
17    189.357000
18    202.321000
19           NaN
Name: Acre - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Acre - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
0,-2.030084,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-1.440003,-1.375240,-1.497951,-1.992410,-1.347524,0.839702,-0.376361,-0.817511,-1.693187,0.511437,-0.302580,-2.159170
1,-1.613449,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-1.350309,-1.227715,-1.202188,-1.728745,-1.075350,0.845730,-0.023671,-0.852793,-1.461874,0.708925,-0.019411,-1.679233
2,-1.238679,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-1.206082,-0.988206,-0.733147,-1.297851,-0.608374,0.880827,0.517116,-0.872285,-1.244099,0.658213,0.320653,-1.246364
3,-0.904868,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-0.998670,-0.715394,-0.249578,-0.808021,-0.069137,0.927709,0.983378,-0.920083,-1.045681,0.892404,0.699688,-0.897688
4,-0.611111,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-0.719418,-0.467961,0.089767,-0.367549,0.419818,0.969095,1.112487,-0.926282,-0.857815,1.061182,1.154652,-0.549083
5,-0.356500,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.359675,-0.304591,0.126134,-0.084728,0.735950,0.987703,0.641815,-0.943019,-0.706882,1.247415,1.289755,-0.399492
6,-0.140130,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.060125,-0.266616,0.156116,0.004937,0.727369,0.823693,-0.092244,-0.926308,-0.601715,1.403496,1.057021,-0.428509
7,0.038904,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,0.552448,-0.238525,0.289409,-0.060227,-1.206975,-1.458847,-1.071249,-0.943239,-0.209591,0.794893,0.837109,0.086954
8,0.181510,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,1.301184,-0.133859,0.584494,-0.094463,-2.068051,-2.369109,-1.132843,-0.951378,0.152718,0.311602,0.724917,0.512197
9,0.268191,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.781405,-0.041160,0.104633,0.062787,-1.106412,-1.384207,0.092186,0.153806,0.491862,0.009045,0.672628,0.926179


In [11]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      58.142000
1      80.914000
2     114.415000
3     121.872000
4     119.145000
5     121.146000
6     174.705000
7     178.322000
8     173.494000
9     193.672000
10    171.709286
11    151.486629
12    134.223086
13    150.491000
14    148.696000
15    153.806000
16    178.033000
Name: Acre - Consumo de Cimento (t), dtype: float64

In [12]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Acre - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
17,0.972756,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.19542,1.95738,0.625166,2.314567,1.380715,-1.527718,-2.588727,1.035224,0.76835,-0.88731,-1.365677,0.306758


In [13]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    189.357
Name: Acre - Consumo de Cimento (t), dtype: float64

In [14]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [18]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [19]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[955932495, 540719117, 2611398158, 3282873654, 3307907219, 4185233443, 1236775872, 4108624331, 3279020349, 3132996652, 2994169300, 1514407316, 892182274, 3754542278, 737148555, 1656991078, 2424397658, 1062413291, 4068803078, 2878773289, 3734173904, 107311958, 2093062067, 3790918950, 2114060176, 1942916510, 2671801531, 1186670764, 3001524760, 808045110, 1023008593, 649738644, 4288535760, 945642312, 2219779145, 1271324517, 3586243935, 1310831427, 2418849591, 43594431, 1638928939, 198839081, 3905877189, 2402779877, 4160207670, 4012828556, 1185454353, 302132563, 2839361329, 1538113004, 3341048574, 102641439, 2414901310, 1450905580, 784526456, 2507822823, 2430435658, 1611967322, 86978181, 605650123, 2655399908, 2574423413, 3913884174, 1405354615, 841643000, 4229809606, 2440502943, 3735120013, 3610648854, 450741745, 1231423480, 1573606478, 3260825071, 3869711572, 1026202172, 2974236398, 1423947706, 145835903, 1105990581, 3515235287, 4253484906, 3074812429, 2564299988, 2191895484, 2107321102,

2023-10-03 18:29:30.999978: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-03 18:29:31.028445: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-03 18:29:31.324380: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-03 18:29:31.356657: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 0.8263249397277832


Step: 17 ___________________________________________
val_loss: 34.35273361206055


Step: 18 ___________________________________________
val_loss: 3.405555009841919


Step: 19 ___________________________________________
val_loss: 31.67458152770996


Step: 20 ___________________________________________
val_loss: 163.74668884277344


Step: 21 ___________________________________________
val_loss: 163.7572021484375


Step: 22 ___________________________________________
val_loss: 164.04859924316406


Step: 23 ___________________________________________
val_loss: 0.16020172834396362


Step: 24 ___________________________________________
val_loss: 163.74652099609375


Step: 25 ___________________________________________
val_loss: 222.90240478515625


Step: 26 ___________________________________________
val_loss: 142.8577117919922


Step: 27 ___________________________________________
val_loss: 162.43930053710938


Step: 28 ________________________________________

2023-10-03 18:37:44.102540: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 0.40553757548332214


Step: 46 ___________________________________________
val_loss: 5.268050193786621


Step: 47 ___________________________________________
val_loss: 101.83163452148438


Step: 48 ___________________________________________
val_loss: 190.8642578125


Step: 49 ___________________________________________
val_loss: 169.2533721923828


Step: 50 ___________________________________________
val_loss: 163.7467041015625


Step: 51 ___________________________________________
val_loss: 0.00536167249083519


Step: 52 ___________________________________________
val_loss: 163.7666778564453


Step: 53 ___________________________________________
val_loss: 0.0029285813216120005


Step: 54 ___________________________________________
val_loss: 0.12077593058347702


Step: 55 ___________________________________________
val_loss: 167.8575439453125


Step: 56 ___________________________________________
val_loss: 163.78533935546875


Step: 57 _______________________________________

...

val_loss: 1.6904975175857544


Step: 77 ___________________________________________
val_loss: 1.331954836845398


Step: 78 ___________________________________________
val_loss: 169.25152587890625


Step: 79 ___________________________________________
val_loss: 163.74676513671875


Step: 80 ___________________________________________
val_loss: 163.37939453125


Step: 81 ___________________________________________
val_loss: 2.168769121170044


Step: 82 ___________________________________________


2023-10-03 18:48:39.329554: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-03 18:48:39.362074: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 163.74961853027344


Step: 83 ___________________________________________
val_loss: 163.69993591308594


Step: 84 ___________________________________________
val_loss: 169.2707977294922


Step: 85 ___________________________________________
val_loss: 163.74688720703125


Step: 86 ___________________________________________
val_loss: 0.0014470393070951104


Step: 87 ___________________________________________
val_loss: 169.26004028320312


Step: 88 ___________________________________________
val_loss: 0.12628604471683502


Step: 89 ___________________________________________


2023-10-03 18:50:33.196661: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-03 18:50:33.229571: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 0.06740199774503708


Step: 90 ___________________________________________
val_loss: 208.2532958984375


Step: 91 ___________________________________________
val_loss: 163.80715942382812


Step: 92 ___________________________________________
val_loss: 2.8632590770721436


Step: 93 ___________________________________________
val_loss: 7.495461940765381


Step: 94 ___________________________________________
val_loss: 167.90451049804688


Step: 95 ___________________________________________
val_loss: 92.06739044189453


Step: 96 ___________________________________________
val_loss: 245.3599090576172


Step: 97 ___________________________________________
val_loss: 169.27159118652344


Step: 98 ___________________________________________
val_loss: 171.6637420654297


Step: 99 ___________________________________________
val_loss: 166.8924560546875


final_seed: 2418849591


In [20]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 456ms/step - loss: 21599.9590 - val_loss: 22319.6191
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 19552.1035 - val_loss: 53951.7969
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 21242.1641 - val_loss: 18455.1465
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 19815.2070 - val_loss: 14963.4854
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 19793.3613 - val_loss: 24675.3848
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 20148.0645 - val_loss: 10645.8818
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 15712.5938 - val_loss: 19149.0508
Epoch 8/10000
1/1 [==============================] - 0s 13ms/step - loss: 15030.0791 - val_loss: 16379.0342
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 8219.2109 - val_loss: 11562.4971
Epoch 10/10000
1/1 [========

1/1 [==============================] - 0s 14ms/step - loss: 442.6094 - val_loss: 163.9780
Epoch 79/10000
1/1 [==============================] - 0s 13ms/step - loss: 166.2092 - val_loss: 168.0961
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 216.6752 - val_loss: 116.6478
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 436.6973 - val_loss: 204.2754
Epoch 82/10000
1/1 [==============================] - 0s 13ms/step - loss: 223.6666 - val_loss: 150.6061
Epoch 83/10000
1/1 [==============================] - 0s 13ms/step - loss: 176.4180 - val_loss: 137.2101
Epoch 84/10000
1/1 [==============================] - 0s 14ms/step - loss: 118.2619 - val_loss: 152.7185
Epoch 85/10000
1/1 [==============================] - 0s 13ms/step - loss: 111.5139 - val_loss: 162.5854
Epoch 86/10000
1/1 [==============================] - 0s 13ms/step - loss: 125.0727 - val_loss: 168.6521
Epoch 87/10000
1/1 [==============================] - 0s 14ms/step - l

1/1 [==============================] - 0s 13ms/step - loss: 29.2293 - val_loss: 4.7089
Epoch 236/10000
1/1 [==============================] - 0s 13ms/step - loss: 23.9776 - val_loss: 5.1361
Epoch 237/10000
1/1 [==============================] - 0s 13ms/step - loss: 23.8488 - val_loss: 4.8656
Epoch 238/10000
1/1 [==============================] - 0s 13ms/step - loss: 22.5963 - val_loss: 3.9963
Epoch 239/10000
1/1 [==============================] - 0s 13ms/step - loss: 22.3902 - val_loss: 3.1472
Epoch 240/10000
1/1 [==============================] - 0s 14ms/step - loss: 14.7035 - val_loss: 2.7251
Epoch 241/10000
1/1 [==============================] - 0s 13ms/step - loss: 20.2901 - val_loss: 3.2106
Epoch 242/10000
1/1 [==============================] - 0s 13ms/step - loss: 21.5357 - val_loss: 3.1150
Epoch 243/10000
1/1 [==============================] - 0s 14ms/step - loss: 21.6058 - val_loss: 4.3252
Epoch 244/10000
1/1 [==============================] - 0s 13ms/step - loss: 20.7029 - val

1/1 [==============================] - 0s 14ms/step - loss: 10.4649 - val_loss: 2.6852
Epoch 395/10000
1/1 [==============================] - 0s 13ms/step - loss: 6.1963 - val_loss: 2.7282
Epoch 396/10000
1/1 [==============================] - 0s 13ms/step - loss: 4.7902 - val_loss: 1.6433
Epoch 397/10000
1/1 [==============================] - 0s 14ms/step - loss: 20.2649 - val_loss: 1.4388
Epoch 398/10000
1/1 [==============================] - 0s 13ms/step - loss: 11.7162 - val_loss: 1.5364
Epoch 399/10000
1/1 [==============================] - 0s 13ms/step - loss: 9.4764 - val_loss: 1.6207
Epoch 400/10000
1/1 [==============================] - 0s 13ms/step - loss: 20.4324 - val_loss: 1.3010
Epoch 401/10000
1/1 [==============================] - 0s 13ms/step - loss: 10.9428 - val_loss: 5.8610
Epoch 402/10000
1/1 [==============================] - 0s 13ms/step - loss: 5.2399 - val_loss: 5.3322
Epoch 403/10000
1/1 [==============================] - 0s 13ms/step - loss: 3.4259 - val_loss

1/1 [==============================] - 0s 13ms/step - loss: 1.5647 - val_loss: 0.8543
Epoch 555/10000
1/1 [==============================] - 0s 14ms/step - loss: 1.0155 - val_loss: 0.7801
Epoch 556/10000
1/1 [==============================] - 0s 13ms/step - loss: 4.9999 - val_loss: 0.8146
Epoch 557/10000
1/1 [==============================] - 0s 14ms/step - loss: 2.0604 - val_loss: 0.6993
Epoch 558/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.8263 - val_loss: 0.7735
Epoch 559/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.7047 - val_loss: 0.8542
Epoch 560/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.5370 - val_loss: 0.7384
Epoch 561/10000
1/1 [==============================] - 0s 13ms/step - loss: 2.1792 - val_loss: 0.8886
Epoch 562/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.6144 - val_loss: 4.0999
Epoch 563/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9346 - val_loss: 0.7

...

1/1 [==============================] - 0s 13ms/step - loss: 0.5041 - val_loss: 0.0120
Epoch 2931/10000
1/1 [==============================] - 0s 13ms/step - loss: 2.9265 - val_loss: 0.0257
Epoch 2932/10000
1/1 [==============================] - 0s 14ms/step - loss: 13.2231 - val_loss: 0.0034
Epoch 2933/10000
1/1 [==============================] - 0s 14ms/step - loss: 17.3552 - val_loss: 0.0065
Epoch 2934/10000
1/1 [==============================] - 0s 14ms/step - loss: 41.0217 - val_loss: 0.4535
Epoch 2935/10000
1/1 [==============================] - 0s 15ms/step - loss: 18.3379 - val_loss: 0.3374
Epoch 2936/10000
1/1 [==============================] - 0s 15ms/step - loss: 12.8644 - val_loss: 0.1413
Epoch 2937/10000
1/1 [==============================] - 0s 14ms/step - loss: 11.2008 - val_loss: 0.1072
Epoch 2938/10000
1/1 [==============================] - 0s 13ms/step - loss: 10.5523 - val_loss: 0.0693
Epoch 2939/10000
1/1 [==============================] - 0s 13ms/step - loss: 16.726

Epoch 3009/10000
1/1 [==============================] - 0s 13ms/step - loss: 14.0674 - val_loss: 0.2946
Epoch 3010/10000
1/1 [==============================] - 0s 13ms/step - loss: 9.4484 - val_loss: 1.2241
Epoch 3011/10000
1/1 [==============================] - 0s 13ms/step - loss: 8.0263 - val_loss: 0.3584
Epoch 3012/10000
1/1 [==============================] - 0s 13ms/step - loss: 10.0604 - val_loss: 0.4421
Epoch 3013/10000
1/1 [==============================] - 0s 13ms/step - loss: 8.4813 - val_loss: 0.0289
Epoch 3014/10000
1/1 [==============================] - 0s 13ms/step - loss: 19.8809 - val_loss: 0.0748
Epoch 3015/10000
1/1 [==============================] - 0s 13ms/step - loss: 3.2646 - val_loss: 0.0596
Epoch 3016/10000
1/1 [==============================] - 0s 13ms/step - loss: 4.7542 - val_loss: 0.0129
Epoch 3017/10000
1/1 [==============================] - 0s 14ms/step - loss: 2.6104 - val_loss: 0.0189
Epoch 3018/10000
1/1 [==============================] - 0s 13ms/step -

1/1 [==============================] - 0s 13ms/step - loss: 1.3216 - val_loss: 1.8487
Epoch 3168/10000
1/1 [==============================] - 0s 13ms/step - loss: 8.9516 - val_loss: 1.1719
Epoch 3169/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.7888 - val_loss: 0.8018
Epoch 3170/10000
1/1 [==============================] - 0s 13ms/step - loss: 5.9934 - val_loss: 1.1780
Epoch 3171/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.8386 - val_loss: 0.6098
Epoch 3172/10000
1/1 [==============================] - 0s 13ms/step - loss: 2.4222 - val_loss: 0.7400
Epoch 3173/10000
1/1 [==============================] - 0s 13ms/step - loss: 1.4650 - val_loss: 1.0118
Epoch 3174/10000
1/1 [==============================] - 0s 13ms/step - loss: 1.7157 - val_loss: 0.9870
Epoch 3175/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.8040 - val_loss: 1.4014
Epoch 3176/10000
1/1 [==============================] - 0s 13ms/step - loss: 2.3873 - val_

1/1 [==============================] - 0s 13ms/step - loss: 0.5752 - val_loss: 0.2522
Epoch 3326/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.4430 - val_loss: 0.2235
Epoch 3327/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.6899 - val_loss: 0.2230
Epoch 3328/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.3340 - val_loss: 0.2120
Epoch 3329/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.5482 - val_loss: 0.2164
Epoch 3330/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.4352 - val_loss: 0.2086
Epoch 3331/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.3541 - val_loss: 0.2098
Epoch 3332/10000
1/1 [==============================] - 0s 13ms/step - loss: 7.4934 - val_loss: 0.2216
Epoch 3333/10000
1/1 [==============================] - 0s 13ms/step - loss: 3.5405 - val_loss: 0.4243
Epoch 3334/10000
1/1 [==============================] - 0s 13ms/step - loss: 2.0756 - val_

1/1 [==============================] - 0s 14ms/step - loss: 0.1525 - val_loss: 0.1429
Epoch 3484/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.4114 - val_loss: 0.1268
Epoch 3485/10000
1/1 [==============================] - 0s 22ms/step - loss: 4.4004 - val_loss: 0.1272
Epoch 3485: early stopping


In [21]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 47ms/step
[189.357](test_target) - [[178.06451]](prediction) = 11.29248583984375


In [22]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [23]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Acre - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
0,-1.719426,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-1.136917,-1.602513,-1.749605,-1.614179,-1.281299,0.304424,-0.850064,1.890129,-1.579970,-1.390443,-1.728144,-1.794558
1,-1.108757,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-0.999535,-1.253417,-1.287691,-1.259683,-0.947419,0.312142,-0.339986,1.083550,-1.069243,-0.701037,-1.203190,-1.105592
2,-0.559449,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.778627,-0.686655,-0.555158,-0.680349,-0.374573,0.357081,0.442130,0.637946,-0.588405,-0.878069,-0.572763,-0.484192
3,-0.070177,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.460940,-0.041086,0.200066,-0.021777,0.286918,0.417111,1.116463,-0.454770,-0.150307,-0.060537,0.129913,0.016343
4,0.360389,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,-0.033220,0.544426,0.730044,0.570435,0.886727,0.470103,1.303186,-0.596500,0.264492,0.528645,0.973348,0.516778
5,0.733577,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,0.517786,0.931017,0.786840,0.950686,1.274530,0.493929,0.622476,-0.979118,0.597744,1.178761,1.223809,0.731521
6,1.050714,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,0.976597,1.020878,0.833666,1.071240,1.264003,0.283926,-0.439158,-0.597097,0.829948,1.723617,0.792356,0.689866
7,1.313129,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,1.914856,1.087351,1.041838,0.983627,-1.108886,-2.638717,-1.855048,-0.984140,1.695740,-0.400936,0.384671,1.429832


train_target:


,Acre - Consumo de Cimento (t)
0,58.142
1,80.914
2,114.415
3,121.872
4,119.145
5,121.146
6,174.705
7,178.322


test_input:


...

,Acre - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
17,0.972756,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.19542,1.95738,0.625166,2.314567,1.380715,-1.527718,-2.588727,1.035224,0.76835,-0.88731,-1.365677,0.306758


test_target:


,Acre - Consumo de Cimento (t)
17,189.357


1/1 [==============================] - 0s 34ms/step
Error: 11.29248583984375




[153.552734375,
 161.91592407226562,
 168.26705932617188,
 179.30455017089844,
 162.43359375,
 164.63705444335938,
 149.9971923828125,
 155.06039428710938,
 122.4489974975586,
 178.06451416015625]

In [24]:
display(targets)
display(predictions)

[173.494,
 193.672,
 171.7092857142857,
 151.48662857142855,
 134.2230857142857,
 150.491,
 148.696,
 153.80599999999998,
 178.033,
 189.357]

[153.552734375,
 161.91592407226562,
 168.26705932617188,
 179.30455017089844,
 162.43359375,
 164.63705444335938,
 149.9971923828125,
 155.06039428710938,
 122.4489974975586,
 178.06451416015625]

In [25]:
mae = mean_absolute_error(predictions, targets)
mae

19.474612703159877

In [26]:
porcentage = mae/np.mean(targets)
porcentage

0.11838900637070071